# Actividad: Little Scheme in Python

En esta tarea uste debe mejorar el lenguaje implementado en clases extendiendo la gramatica. Ademas de implementar dos visitors que detected anomalias en código escrito en nuestro lenguaje.

## Modelo de Clases

Agregue clases al modelo para soportar la gramatica solicitada.


In [1]:
from abc import ABC, abstractmethod

# -----------
class Node:

  def pretty_print(self):
    self.print_tree(0)

  def print_tree(self, level):
    pass

  def eval(self):
    pass

  def accept(self, visitor):
    pass

# -----------
class LiteralNode(Node):
  def __init__(self, val):
      self.value = val

# -----------

class NumberNode(LiteralNode):
  def __init__(self, val):
      self.value = val

  def eval(self):
    return self.value

  def print_tree(self, level):
    print(' ' * level + str(self.value))

  def accept(self, visitor):
    visitor.visit_number(self)

# -----------
class OperatorNode(Node):
  def __init__(self, sym):
      self.symbol = sym

# -----------
class BinaryOperatorNode(OperatorNode):
  def __init__(self, leftNode, rightNode, sym):
    super().__init__(sym)
    self.leftNode = leftNode
    self.rightNode = rightNode

  def print_tree(self, level):
    print(' ' * level + self.symbol)
    self.leftNode.print_tree(level + 1)
    self.rightNode.print_tree(level + 1)

# -----------
class AdditionNode(BinaryOperatorNode):
  def __init__(self, leftNode, rightNode):
      super(AdditionNode, self).__init__(leftNode, rightNode, "+")
  def eval(self):
    return self.leftNode.eval() + self.rightNode.eval()

  def accept(self, visitor):
    visitor.visit_addition(self)

# -----------

class SubstractNode(BinaryOperatorNode):
  def __init__(self, leftNode, rightNode):
      super(SubstractNode, self).__init__(leftNode, rightNode, "-")

  def eval(self):
    return self.leftNode.eval() - self.rightNode.eval()

  def accept(self, visitor):
    visitor.visit_substraction(self)







## Parsing
Modifique el parser para que pueda instanciar objetos de las nuevas clases creadas.

In [2]:
# Parser super sencillo y basico
def parser(stringCode):
    # si es número, entonces creo un NumberNode
    if isNumber(stringCode):
        return NumberNode(int(stringCode))
    # si es operación llamo al método operation
    elif isOperation(stringCode):
        return operation(stringCode)
    # si no es ningúno el string no sigue nuestra gramatica.
    else:
        raise Exception("Expression ", stringCode, " not supported.")


def isNumber(string):
    return string.isnumeric()


def isOperation(string):
    res = False
    validOperators = {'+': 2, '-': 2, '++': 1}
    # Primero verificamos si tiene ambos parentesis de la sintaxis
    if string[0] == '(' and string[len(string)-1] == ')':
        # Partimos por espacios la cadena sin considerar el primer y el último parentesis
        tokens = splitArgs(string[1:len(string)-1])
        operator = tokens.pop(0)
        # Verificamos que el operador sea uno que reconocemos y que solo exista el número correcto de argumentos
        res = operator in validOperators and len(tokens) == validOperators.get(operator)
    return res


def operation(string):
    tokens = splitArgs(string[1:len(string)-1])
    operator = tokens.pop(0)
    # Creamos el nodo correspondiente segun el operador y parseamos los demás argumentos
    if operator == "+":
        return AdditionNode(parser(tokens[0]), parser(tokens[1]))
    if operator == "-":
        return SubstractNode(parser(tokens[0]), parser(tokens[1]))
    if operator == "++":
        return AdditionNode(parser(tokens[0]), NumberNode(1))

def splitArgs(string):
    open_counter = 0
    temp = ""
    result = []
    for key in string:
        if key == ' ' and open_counter == 0:
            # adding an argument
            result.append(temp.strip())
            temp = ""
        elif key == '(':
            open_counter = open_counter + 1
        elif key == ')':
            open_counter = open_counter - 1
        else:
            pass
        temp = temp + key
    # adding last arg
    result.append(temp.strip())
    return result


# Progando el Paser

In [ ]:
expr = "(if0 (- 1 1) (print (+ 3 3)) (print (+ 2 2)))"
node = parser(expr)
node.pretty_print()

In [ ]:
expr = "(double (abs 1))"
node = parser(expr)
node.pretty_print()

## Probando el Eval

In [ ]:
expr = "(if0 (- 1 1) (print (+ 3 3)) (print (+ 2 2)))"
node = parser(expr)
node.eval()

In [ ]:
expr = "(if0 (print (+ 1 1)) (print (+ 3 3)) (print (+ 2 2)))"
node = parser(expr)
node.eval()

In [ ]:
expr = "(double (abs (- 0 12)))"
node = parser(expr)
node.eval()

In [ ]:
expr = "(double (- 0 12))"
node = parser(expr)
node.eval()

## Actualizando la clase Visitor

Actualize la clase CodeVisitor para que soporte los nuevos tipos de nodos.

In [5]:
class CodeVisitor:
  def visit_number(self, node: NumberNode):
    pass
  def visit_addition(self, node: AdditionNode):
    node.leftNode.accept(self)
    node.rightNode.accept(self)

  def visit_substraction(self, node: SubstractNode):
    node.leftNode.accept(self)
    node.rightNode.accept(self)



In [6]:
class If0CounterVisitor(CodeVisitor):
  pass
  # complete la clase

In [7]:
class RedundantAbsVisitor(CodeVisitor):
  pass
  # complete la clase

## Visitor Tests

In [ ]:
expr = "(if0 0 1 2)"
tree = parser(expr)
visitor = If0CounterVisitor()
tree.accept(visitor)
assert visitor.total() == 1

In [ ]:
expr = "(if0 0 (if0 1 1 2) (if0 0 1 1))"
tree = parser(expr)
visitor = If0CounterVisitor()
tree.accept(visitor)
assert visitor.total() == 3

In [ ]:
expr = "(if0 (if0 1 1 2) (if0 1 1 2) (if0 0 1 1))"
tree = parser(expr)
visitor = If0CounterVisitor()
tree.accept(visitor)
assert visitor.total() == 3

In [ ]:
expr = "(+ 1 (- 1 (if0 1 1 2)))"
tree = parser(expr)
visitor = If0CounterVisitor()
tree.accept(visitor)
assert visitor.total() == 1

In [ ]:
expr = "(print (if0 1 1 2))"
tree = parser(expr)
visitor = If0CounterVisitor()
tree.accept(visitor)
assert visitor.total() == 1

In [ ]:
expr = "(abs (abs 2))"
tree = parser(expr)
visitor = RedundantAbsVisitor()
tree.accept(visitor)
assert visitor.total() == 2

In [ ]:
expr = "(+ (abs (- 1 2)) (abs 2))"
tree = parser(expr)
visitor = RedundantAbsVisitor()
tree.accept(visitor)
assert visitor.total() == 1

In [ ]:
expr = "(+ (abs (+ 1 2)) (abs 2))"
tree = parser(expr)
visitor = RedundantAbsVisitor()
tree.accept(visitor)
# sigue siendo uno pq un analizador sintactico no evalua
# por lo que no sabe que la operacion dentro del abs da positivo
assert visitor.total() == 1

In [ ]:
expr = "(double (abs 1))"
tree = parser(expr)
visitor = RedundantAbsVisitor()
tree.accept(visitor)
#porque double esta implementado con azucar sintactico
assert visitor.total() == 2